In [1]:
import os

import tensorflow.keras.backend as ktf
from data_loader import *
from network import *
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, TensorBoard, EarlyStopping

import cfg

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

0123456789.-|
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '.', 11: '-', 12: '|'}


In [5]:
train_model, infer_model = CRNN(cfg.width, cfg.height, cfg.label_len, cfg.characters).network()

if cfg.load_pre_model:
    train_model.load_weights(cfg.load_pre_model_path, by_name=True, skip_mismatch=True)


#train_model.load_weights(cfg.saved_model_weights_path)
train_model.load_weights("saved_model//BLSTM_E66_L0.112.h5")
train_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 32, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 200, 32, 64)  1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 200, 32, 64)  256         conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 200, 32, 128) 73856       batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [6]:
train_model.fit(img_gen_train(batch_size=50, input_shape=(None, 50)),
                          steps_per_epoch=300, epochs=100, initial_epoch=66,verbose=1,
                          callbacks=[ModelCheckpoint(cfg.checkpoint_path, monitor='loss', save_best_only=True, save_weights_only=True),
                                     EarlyStopping(monitor='loss',patience=5),
                                     #Evaluate(),
                                     TensorBoard(log_dir=cfg.log_dir)
                                    ]
                          )

infer_model.save(cfg.saved_model_path)
infer_model.save_weights(cfg.saved_model_weights_path)


  ...
    to  
  ['...']
Train for 300 steps
Epoch 67/100
300/300 [==============================] - 376s 1s/step - loss: 0.1328
Epoch 68/100
300/300 [==============================] - 373s 1s/step - loss: 0.1306
Epoch 69/100
300/300 [==============================] - 371s 1s/step - loss: 0.1373
Epoch 70/100
300/300 [==============================] - 371s 1s/step - loss: 0.1375
Epoch 71/100
300/300 [==============================] - 371s 1s/step - loss: 0.1350
Epoch 72/100
300/300 [==============================] - 372s 1s/step - loss: 0.1219
Epoch 73/100
300/300 [==============================] - 373s 1s/step - loss: 0.1096
Epoch 74/100
300/300 [==============================] - 373s 1s/step - loss: 0.1305
Epoch 75/100
300/300 [==============================] - 372s 1s/step - loss: 0.1924
Epoch 76/100
300/300 [==============================] - 371s 1s/step - loss: 0.2145
Epoch 77/100
300/300 [==============================] - 371s 1s/step - loss: 0.1442
Epoch 78/100
300/300 [=========

In [ ]:

import os
import cv2
import numpy as np
import tensorflow.keras.backend as ktf
from data_loader import *
from network import CRNN

import cfg



def predict(infer_model, img_path):
    img = cv2.imread(img_path)
    img_size = img.shape
    if (img_size[1] / img_size[0] * 1.0) < 6:
        img_reshape = cv2.resize(img, (int(31.0 / img_size[0] * img_size[1]), cfg.height))

        mat_ori = np.zeros((cfg.height, cfg.width - int(31.0 / img_size[0] * img_size[1]), 3), dtype=np.uint8)
        out_img = np.concatenate([img_reshape, mat_ori], axis=1).transpose([1, 0, 2])
    else:
        out_img = cv2.resize(img, (cfg.width, cfg.height), interpolation=cv2.INTER_CUBIC)
        out_img = np.asarray(out_img)
        out_img = out_img.transpose([1, 0, 2])

    y_pred = infer_model.predict(np.expand_dims(out_img, axis=0))
    shape = y_pred[:, 2:, :].shape
    ctc_decode = ktf.ctc_decode(y_pred[:, 2:, :], input_length=np.ones(shape[0]) * shape[1])[0][0]
    out = ktf.get_value(ctc_decode)[:, :cfg.label_len]
    result = label_to_string(out[0])
    return result



_, model = CRNN(cfg.width, cfg.height, cfg.label_len, cfg.characters).network()
model.load_weights(cfg.saved_model_weights_path)
#model.load_weights('saved_model/weights_E10_L0.187.h5')

# model.summary()

imgs_path='test_imgs'
result_path='result.txt'

imgs_list = os.listdir(imgs_path)
result_txt = open(result_path, 'w+')

for img_name in imgs_list:
    result = predict(model, os.path.join(imgs_path, img_name))
    result = img_name + " : " + result + "\n"
    print(result)
    result_txt.write(result)
result_txt.close()

